In [1]:
import numpy as np
import sympy
import csv
from json import loads

lab_number = '2.2.2'


def toCopyString(matrix) -> str:
    return 'matrix(' + str(matrix).replace('Matrix([', '[').replace('])', ']')[1:-1].replace(' ', '') + ')'

def fromCopyString(string: str):
    return loads(string.replace('matrix(', '[').replace(')', ']'))

def получить_матрицы_варианта(number):
    filename = f'./{lab_number}_variants.txt'
    with open(filename, 'r') as file:
        variants = csv.reader(file, delimiter=',')
        variant = list(variants)[number - 1][1].split(';')
        A = fromCopyString(variant[0])
        B = fromCopyString(variant[1])
        L = fromCopyString(variant[2])
        C = fromCopyString(variant[3])
    return A, B, L, C

In [96]:
A, B, L, C = получить_матрицы_варианта(13)
A = sympy.Matrix(A)
B = sympy.Matrix(B)
L = sympy.Matrix(L)
C = sympy.Matrix(C)
C

Matrix([
[ 1, -3, 0],
[-2, -1, 0]])

In [97]:
def матрица_управляемости(A, B, L):
    матрица = sympy.zeros(3, 6)
    матрица[:, 0:2] = B 
    матрица[:, 2:4] = A*B
    матрица[:, 4:] = A**2*B
    return матрица

In [98]:
S = матрица_управляемости(A, B, L)
S

Matrix([
[ 2,  2,  20,  14,  140,  94],
[-4, -2, -19, -12, -118, -78],
[ 3,  2,  21,  14,  141,  94]])

In [99]:
toCopyString(S)

'matrix([2,2,20,14,140,94],[-4,-2,-19,-12,-118,-78],[3,2,21,14,141,94])'

In [100]:
def определитель_матрицы_управляемости(S):
    определитель = S[:, :3].det()
    # print('Да!' if определитель != 0 else 'Нет :(')
    return определитель

In [101]:
print(определитель_матрицы_управляемости(S))

6


In [102]:
def матрица_желаемых_коэффициентов(L):
    res = sympy.zeros(3, 1)
    res[2, 0] = -sum(L)
    res[1, 0] = L[0]*L[1] + L[1]*L[2] + L[2]*L[0]
    res[0, 0] = -L[0]*L[1]*L[2]
    return res

In [103]:
a = матрица_желаемых_коэффициентов(L)
a

Matrix([
[4],
[8],
[5]])

In [104]:
def вектор_d(A, B, L, C):
    q = sympy.Matrix([sympy.symbols('q_1'), 1])
    s = sympy.symbols('s')
    tmp = (s*sympy.eye(3) - A).inv()*B*q
    tmp = tmp.applyfunc(lambda x: sympy.simplify(x))
    highest_denominator = sympy.fraction(tmp[0, 0])[1]
    for i in range(3):
        if not sympy.div(sympy.fraction(tmp[i, 0])[1], highest_denominator)[1]:
            highest_denominator = sympy.fraction(tmp[i, 0])[1]
    # print(highest_denominator)
    return sympy.Matrix(sympy.Poly(highest_denominator).all_coeffs()[:0:-1])

In [105]:
d = вектор_d(A, B, L, C)
d

Matrix([
[-9],
[17],
[-9]])

In [106]:
def матрица_g_q(A, B, L, C):
    q = sympy.Matrix([sympy.symbols('q_1'), 1])
    s = sympy.symbols('s')
    tmp = (s*sympy.eye(3) - A).inv()*B*q
    tmp = tmp.applyfunc(lambda x: sympy.simplify(x))
    # multiply every element in tmp by denominator of tmp[0, 0]
    tmp = tmp.applyfunc(lambda x: sympy.simplify(x*sympy.fraction(tmp[0, 0])[1]))
    tmp = tmp.applyfunc(lambda x: sympy.simplify(x*sympy.fraction(tmp[1, 0])[1]))
    tmp = tmp.applyfunc(lambda x: sympy.simplify(x*sympy.fraction(tmp[2, 0])[1]))
    
    return tmp    

In [107]:
gq = матрица_g_q(A, B, L, C)
gq

Matrix([
[   2*q_1*s**2 + 2*q_1*s - 6*q_1 + 2*s**2 - 4*s + 2],
[-4*q_1*s**2 + 17*q_1*s - 15*q_1 - 2*s**2 + 6*s - 4],
[   3*q_1*s**2 - 6*q_1*s + 3*q_1 + 2*s**2 - 4*s + 2]])

In [108]:
def матрица_L_q(A, B, L, C):
    s = sympy.symbols('s')
    tmp = матрица_g_q(A, B, L, C)
    res = sympy.zeros(3, 3)
    res[0] = sympy.Poly(tmp[0, 0], s).all_coeffs()[::-1]
    res[1] = sympy.Poly(tmp[1, 0], s).all_coeffs()[::-1]
    res[2] = sympy.Poly(tmp[2, 0], s).all_coeffs()[::-1]
    return res.T

In [109]:
matrix_L = матрица_L_q(A, B, L, C)
matrix_L

Matrix([
[  2 - 6*q_1,  2*q_1 - 4,  2*q_1 + 2],
[-15*q_1 - 4, 17*q_1 + 6, -4*q_1 - 2],
[  3*q_1 + 2, -6*q_1 - 4,  3*q_1 + 2]])

In [110]:
toCopyString(matrix_L)

'matrix([2-6*q_1,2*q_1-4,2*q_1+2],[-15*q_1-4,17*q_1+6,-4*q_1-2],[3*q_1+2,-6*q_1-4,3*q_1+2])'

$ \alpha \vec{a} = \beta $

$ \alpha = (I_H - E_H E_B^{-1}I_B)(L^T)^{-1} $

In [111]:
def матрицы_I_E(A, B, L, C):
    res_I  = sympy.zeros(2, 3)
    res_I_ = sympy.zeros(1, 3)
    res_E  = sympy.zeros(2, 2)
    res_E_ = sympy.zeros(1, 2)

    is_first = True
    tmp = [0, 1, 2]
    for i in range(3):
        if C.T[i, 0] or C.T[i, 1]:
            tmp.remove(i)
            if is_first:
                res_E[0, :] = C.T[i, :]
                res_I[0, i] = 1
                is_first = False
            else:
                res_I[1, i] = 1
                res_E[1, :] = C.T[i, :]
    res_I_[tmp[0]] = 1
    res_E_ = C.T[tmp[0], :]
    return res_I, res_I_, res_E, res_E_

In [112]:
Iв, Iн, Eв, Eн = матрицы_I_E(A, B, L, C)
Iв

Matrix([
[1, 0, 0],
[0, 1, 0]])

In [113]:
Iн

Matrix([[0, 0, 1]])

In [114]:
Eв

Matrix([
[ 1, -2],
[-3, -1]])

In [115]:
Eн

Matrix([[0, 0]])

In [116]:
def матрица_alpha_beta(A, B, L, C):
    matrix_L = матрица_L_q(A, B, L, C)
    d = вектор_d(A, B, L, C)
    Iв, Iн, Eв, Eн = матрицы_I_E(A, B, L, C)
    alpha = (Iн - Eн*Eв.inv()*Iв)*(matrix_L.T).inv()
    beta = alpha * d
    return alpha, beta

In [117]:
alpha, beta = матрица_alpha_beta(A, B, L, C)
alpha

Matrix([[(7*q_1 + 1)/(3*q_1**2 + 2*q_1), (9*q_1 + 1)/(3*q_1**2 + 2*q_1), (12*q_1 + 1)/(3*q_1**2 + 2*q_1)]])

In [118]:
beta

Matrix([[-9*(7*q_1 + 1)/(3*q_1**2 + 2*q_1) + 17*(9*q_1 + 1)/(3*q_1**2 + 2*q_1) - 9*(12*q_1 + 1)/(3*q_1**2 + 2*q_1)]])

In [119]:
def вектор_q(A, B, L, C):
    q = sympy.Matrix([sympy.symbols('q_1'), 1])
    alpha, beta = матрица_alpha_beta(A, B, L, C)
    a = матрица_желаемых_коэффициентов(L)
    eqv = alpha * a - beta
    # find q_1 
    q_1 = sympy.solve(eqv[0, 0], q[0])[0]
    q[0] = q_1
    return q.n()

In [120]:
q = вектор_q(A, B, L, C)
q

Matrix([
[-0.101123595505618],
[               1.0]])

In [121]:
toCopyString(q)

'matrix([-0.101123595505618],[1.00000000000000])'

In [145]:
def матрица_g(A, B, L, C):
    q = вектор_q(A, B, L, C)
    s = sympy.symbols('s')
    tmp = (s*sympy.eye(3) - A).inv()*B
    tmp = tmp.applyfunc(lambda x: sympy.simplify(x))
    
    den1 = (sympy.fraction(tmp[0, 0])[1])
    den2 = (sympy.fraction(tmp[1, 0])[1])
    den3 = (sympy.fraction(tmp[2, 0])[1])

    # find the smallest common multiple of all denominators
    # highest_denominator = den1
    # for i in range(2):
    #     highest_denominator = sympy.lcm(highest_denominator, sympy.fraction(tmp[i, 0])[1])

    # print(highest_denominator)
    # for i in range(3):
    #     denominator = sympy.fraction(tmp[i, 0])[1]
    #     if sympy.div(highest_denominator, denominator)[0] != 1:
    #         new_denominator = sympy.lcm(highest_denominator, denominator)
    #         new_numenator = sympy.div(new_denominator, denominator)[0] * sympy.fraction(tmp[i, 0])[0]
    #         tmp[i, 0] = sympy.simplify(new_numenator / new_denominator)

    tmp = tmp.applyfunc(lambda x: sympy.simplify(x*sympy.fraction(tmp[0, 0])[1]))
    tmp = tmp.applyfunc(lambda x: sympy.simplify(x*sympy.fraction(tmp[1, 0])[1]))
    tmp = tmp.applyfunc(lambda x: sympy.simplify(x*sympy.fraction(tmp[2, 0])[1]))

    for i in range(3):
        denominator = sympy.fraction(tmp[i, 0])[1]
        # print(denominator)
        if denominator != 1:
            numerator = sympy.fraction(tmp[i, 0])[0]
            tmp[i, 0] = sympy.div(numerator, denominator)[0]
            
    return tmp*q

In [147]:
g = матрица_g(A, B, L, C)
g

Matrix([
[ 1.79775280898876*s**2 - 4.20224719101124*s + 2.60674157303371],
[-1.59550561797753*s**2 + 4.28089887640449*s - 2.48314606741573],
[ 1.69662921348315*s**2 - 3.39325842696629*s + 1.69662921348315]])

In [148]:
def матрица_L(A, B, L, C):
    s = sympy.symbols('s')
    tmp = матрица_g(A, B, L, C)
    res = sympy.zeros(3, 3)
    res[0] = sympy.Poly(tmp[0, 0], s).all_coeffs()[::-1]
    res[1] = sympy.Poly(tmp[1, 0], s).all_coeffs()[::-1]
    res[2] = sympy.Poly(tmp[2, 0], s).all_coeffs()[::-1]
    return res.T

In [149]:
L_ = матрица_L(A, B, L, C)
L_

Matrix([
[ 2.60674157303371, -4.20224719101124,  1.79775280898876],
[-2.48314606741573,  4.28089887640449, -1.59550561797753],
[ 1.69662921348315, -3.39325842696629,  1.69662921348315]])

In [88]:
def вектор_p(A, B, L, C):
    matrix_L = матрица_L(A, B, L, C)
    a = матрица_желаемых_коэффициентов(L)
    d = вектор_d(A, B, L, C)
    Iв, Iн, Eв, Eн = матрицы_I_E(A, B, L, C)
    p = (Eв).inv()*Iв*(matrix_L.T).inv()*(a-d)
    return p.T

In [150]:
p = вектор_p(A, B, L, C)
p

Matrix([[-5.72563859981078, -25.8495742667928]])

In [151]:
toCopyString(p)

'matrix([-5.72563859981078,-25.8495742667928])'

In [152]:
def матрица_P(A, B, L, C):
    p = вектор_p(A, B, L, C)
    q = вектор_q(A, B, L, C)
    return q * p

In [153]:
P = матрица_P(A, B, L, C)
P

Matrix([
[0.578997161778618,  2.61400189214759],
[-5.72563859981078, -25.8495742667928]])

In [154]:
toCopyString(P)

'matrix([0.578997161778618,2.61400189214759],[-5.72563859981078,-25.8495742667928])'

In [155]:
for var_number in range(1, 31):
    A, B, L, C = получить_матрицы_варианта(var_number)
    A = sympy.Matrix(A)
    B = sympy.Matrix(B)
    L = sympy.Matrix(L)
    C = sympy.Matrix(C)
    try:
        S            = матрица_управляемости(A, B, L)
        определитель = определитель_матрицы_управляемости(S)
        q            = вектор_q(A, B, L, C)
        p            = вектор_p(A, B, L, C)
        P            = матрица_P(A, B, L, C)

        print(var_number)
        with open(f'./results/{var_number}.txt', 'w') as file:
            file.write(toCopyString(S) + '\n')
            file.write(str(определитель) + '\n')
            file.write(toCopyString(q) + '\n')
            file.write(toCopyString(p) + '\n')
            file.write(toCopyString(P) + '\n')
    except:
        pass
        

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
